In [45]:
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [46]:
from missile_gym import MissileGym

In [101]:
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pylab import figure, subplot2grid
from functional import seq
from ipywidgets import interact, interactive, fixed
import ipywidgets
from math import *
%config InlineBackend.figure_format = 'svg'

In [102]:
gym = MissileGym.make('standart')
done = False
reward = 0
obs = gym.reset()
state = gym.get_state()
history = [(state, obs, 0, done, {})]
alphas_targeting = [0]
while not done:
    obs, reward, done, info = gym.step_with_guidance()
    state = gym.get_state()
    history.append((state, obs, reward, done, info))
    alphas_targeting.append(gym.missile.alpha_targeting if abs(gym.missile.alpha_targeting) < gym.missile.alphamax else copysign(gym.missile.alphamax, gym.missile.alpha_targeting))
print(info['done_reason'])

мы попали


In [116]:
def get_an_atau(vel0, vel1, tau):
    g = 9.81
    vel0 = np.array(vel0)
    vel1 = np.array(vel1)
    a = (vel1-vel0)/tau - np.array([0, -g])
    a_tau = np.dot(a, vel0/np.linalg.norm(vel0)) *  vel0/np.linalg.norm(vel0)
    a_n = a - a_tau
    return np.linalg.norm(a_n)/g, copysign(np.linalg.norm(a_tau)/g, np.dot(a_tau, vel0))
    

In [117]:
mis_xs = []
mis_ys = []
trg_xs = []
trg_ys = []
ts = []
mis_vs = []
trg_vs = []
rewards = []
alphas = []
trg_vels = []
mis_vels = []
for state, obs, reward, done, info in tqdm(history):
    gym.set_state(state)
    mis_smry = gym.missile.get_summary()
    trg_smry = gym.target.get_summary()
    mis_xs.append(mis_smry['x'])
    mis_ys.append(mis_smry['y'])
    trg_xs.append(trg_smry['x'])
    trg_ys.append(trg_smry['y'])
    ts.append(mis_smry['t'])
    mis_vs.append(mis_smry['v'])
    trg_vs.append(trg_smry['v'])
    rewards.append(reward)
    alphas.append(mis_smry['alpha'])
    
    trg_vels.append(gym.target.vel)
    mis_vels.append(gym.missile.vel)

trg_nys = [get_an_atau(v0, v1, t1-t0)[0] for v0, v1, t1, t0 in zip(trg_vels, trg_vels[1:], ts[1:], ts)]
trg_nys += [trg_nys[-1]]
mis_nys = [get_an_atau(v0, v1, t1-t0)[0] for v0, v1, t1, t0 in zip(mis_vels, mis_vels[1:], ts[1:], ts)]
mis_nys += [trg_nys[-1]]

trg_nxs = [get_an_atau(v0, v1, t1-t0)[1] for v0, v1, t1, t0 in zip(trg_vels, trg_vels[1:], ts[1:], ts)]
trg_nxs += [trg_nxs[-1]]
mis_nxs = [get_an_atau(v0, v1, t1-t0)[1] for v0, v1, t1, t0 in zip(mis_vels, mis_vels[1:], ts[1:], ts)]
mis_nxs += [trg_nxs[-1]]


In [118]:
@interact(i=ipywidgets.IntSlider(min=0,max=len(history)-1,step=1, value=len(history)-1))
def draw(i):
    f0 = figure(num = 0, figsize = (17, 7))
    ax01 = subplot2grid((4, 7), (0, 0), colspan=4, rowspan=4)
    ax02 = subplot2grid((4, 7), (0, 4), colspan=3, rowspan=1)
    ax03 = subplot2grid((4, 7), (1, 4), colspan=3, rowspan=1,sharex=ax02)
    ax04 = subplot2grid((4, 7), (2, 4), colspan=3, rowspan=1,sharex=ax02)
    ax05 = subplot2grid((4, 7), (3, 4), colspan=3, rowspan=1,sharex=ax02)
    f0.subplots_adjust(hspace=0)   
    f0.subplots_adjust(wspace=0.7)  
    
    state = history[i][0]
    gym.set_state(state)
    gym.render(fig=f0, ax=ax01)
    ax01.plot(mis_xs[:i], mis_ys[:i], ls=':')
    ax01.plot(trg_xs[:i], trg_ys[:i], ls=':')
    ax01.grid(True)
    
    ax02.plot(ts[:i], mis_vs[:i], label='Скорость ракеты')
    ax02.plot(ts[:i], trg_vs[:i], label='Скорость цели')
    ax02.legend()
    ax02.grid(True)
    
    ax03.plot(ts[:i], rewards[:i], label='Награда_')
    ax03.legend()
    ax03.grid(True)
    
    ax04.plot(ts[:i], alphas[:i], label=r'$\alpha_{текущий}$')
    ax04.plot(ts[:i], alphas_targeting[:i], label=r'$\alpha_{потребный}$', color='y', ls='-')
    ax04.legend()
    ax04.grid(True)
    
    ax05.plot(ts[:i], mis_nys[:i], label=r'$n_y$ ракеты', c='b')
    ax05.plot(ts[:i], trg_nys[:i], label=r'$n_y$ цели', c='orange')
    
    ax05.plot(ts[:i], mis_nxs[:i], label=r'$n_x$ ракеты', c='b', ls='--')
    ax05.plot(ts[:i], trg_nxs[:i], label=r'$n_x$ цели', c='orange', ls='--')
    ax05.legend()
    ax05.grid(True)
    
    plt.show()

interactive(children=(IntSlider(value=191, description='i', max=191), Output()), _dom_classes=('widget-interac…

In [93]:
a = np.array([1,0])
b = np.array([1,0])
np.cross(a,b)

array(0)